# LinkedIn Post Generator


---
## 1.&nbsp; Installations and Settings 🛠️

**faiss-cpu** is a library that provides a fast and efficient database for storing and retrieving our numerical summaries of information.

---
## 2.&nbsp; Merged Code 🧠

In [953]:
%%writefile 1_Home.py

from langchain_huggingface import HuggingFaceEndpoint, HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain_community.document_loaders import CSVLoader
from datetime import datetime
import streamlit as st
import os
from Keys__tokens import *
from collections import Counter
import re
import spacy
import base64
import requests
import random
import sqlite3

#Importing keys
HUGGINGFACEHUB_API_TOKEN = "HugFaceWBS"

#from google.colab import userdata # we stored our access token as a colab secret
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN



#llm selection

from langchain_huggingface import HuggingFaceEndpoint

# This info's at the top of each HuggingFace model page
hf_model = "mistralai/Mixtral-8x7B-Instruct-v0.1"

# Create the HuggingFaceEndpoint and add the `add_to_git_credential=True` option
llm = HuggingFaceEndpoint(
    repo_id=hf_model,
    top_k=10,
    temperature=0.01,
    max_new_tokens=250,
    huggingfacehub_api_token = HugFaceWBS
)


#Creating vectors with embeddings
from langchain_huggingface import HuggingFaceEmbeddings


# embeddings
embedding_model = "sentence-transformers/all-MiniLM-l6-v2"
embeddings_folder = "Content"

embeddings = HuggingFaceEmbeddings(model_name=embedding_model,
                                   cache_folder=embeddings_folder)



#Creating a vector database
from langchain.vectorstores import FAISS

#vector_db = FAISS.from_documents(pages, embeddings)

#Once the database is made, you can save it to use over and over again in the future.

#vector_db.save_local("/Users/alexandrealbieri/Documents/Personal/Berlin/WBS Coding School/WBS Bootcamp/Data Science/Generative AI/Content/faiss_index")
#Here's the code to load it again.
vector_db = FAISS.load_local("Content/faiss_index", embeddings, allow_dangerous_deserialization=True)


# memory
@st.cache_resource
def init_memory(_llm):
    return ConversationBufferMemory(
        llm=llm,
        output_key='answer',
        memory_key='chat_history',
        return_messages=True)
memory = init_memory(llm)


#Adding a prompt
from langchain.prompts.prompt import PromptTemplate

input_template = """
You are a LinkedIn post assistant. Create a unique and professional LinkedIn posts using a database of high-quality posts as inspiration, following these guidelines:

Topic and Tone: The user will specify the topic. Use the database only for inspiration, crafting a completely unique post that aligns with the topic and maintains a professional tone.
Length: Keep the post within 200 words. Ensure the content flows naturally and remains concise.
Originality and Anonymity: Do not reference any specific names, places, or identifiable details from the database posts.
Structure: Begin with an engaging opening, followed by 1-2 sentences providing valuable insight, and end with a brief call-to-action or takeaway.
Hashtags: Add 2-4 relevant hashtags based on the post's main themes, placing them at the end to boost visibility.
Produce a LinkedIn post that is polished, unique, and structured, adhering to these guidelines.

Example Post:

"Effective leadership isn’t just about managing tasks—it’s about inspiring and empowering your team to reach new heights. When leaders foster trust and
support growth, they create an environment where innovation thrives, and individuals feel valued. In today’s dynamic work environment, staying adaptable and 
focused on continuous learning can make all the difference. How are you empowering your team to achieve their best? #Leadership #Teamwork #GrowthMindset"

Context to answer question: {context}

Question to be answered: {question}
Response:"""

prompt = PromptTemplate(template=input_template,
                        input_variables=["context", "question"])


# RAG - Chaining it all together
from langchain.chains import RetrievalQA

retriever = vector_db.as_retriever(search_kwargs={"k": 6})

chain = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory,
    return_source_documents=True,
    combine_docs_chain_kwargs={"prompt": prompt}
)

##### Streamlit #####


# Unsplash API credentials
UNSPLASH_ACCESS_KEY = "UNSPLASH_KEY"

# Function to extract hashtags
def extract_hashtags(text):
    return re.findall(r"#(\w+)", text)

# Function to split hashtags into words
def split_hashtag(hashtag):
    return " ".join(re.findall(r'[a-zA-Z][^A-Z]*', hashtag))

# Helper function to make a request to Unsplash API
def make_unsplash_request(query):
    random_seed = random.randint(1, 1000)
    url = f"https://api.unsplash.com/photos/random?query={query}&count=3&client_id={UNSPLASH_ACCESS_KEY}&random_seed={random_seed}"
    response = requests.get(url)
    if response.status_code == 200:
        images = response.json()
        if isinstance(images, list) and len(images) > 0:
            return [img['urls']['small'] for img in images]
    return []

# Function to fetch images from Unsplash
def fetch_images_from_unsplash(keyword):
    images = make_unsplash_request(keyword)
    if images:
        return images
    images = make_unsplash_request("default")
    if images:
        return images
    st.error(f"Failed to retrieve images from Unsplash.")
    return []

# Database setup functions
# Function to get database connection to posts
def get_post_connection():
    try:
        conn = sqlite3.connect("posts.db")
        # Add the new column for date if it doesn't exist
        conn.execute("CREATE TABLE IF NOT EXISTS posts (id INTEGER PRIMARY KEY, content TEXT)")
        # Alter table to add the 'date' column if it does not exist
        try:
            conn.execute("ALTER TABLE posts ADD COLUMN date TEXT")
        except sqlite3.OperationalError:
            # This exception occurs if the column already exists, which is fine
            pass
        return conn
    except sqlite3.Error as e:
        st.error(f"Database connection failed: {e}")
        return None

# Function to save a post to the database
def save_post(content):
    conn = get_post_connection()
    if conn:
        cursor = conn.cursor()
        # Save the content and the current date in the correct format
        current_date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        cursor.execute("INSERT INTO posts (content, created_at) VALUES (?, ?)", (content, current_date))
        conn.commit()
        conn.close()

# Function to get database connection to images
def get_image_connection():
    try:
        conn = sqlite3.connect("images.db")
        conn.execute("CREATE TABLE IF NOT EXISTS images (id INTEGER PRIMARY KEY, url TEXT)")
        return conn
    except sqlite3.Error as e:
        st.error(f"Database connection failed: {e}")
        return None

# Function to save an image to the database
def save_image_to_db(url):
    conn = get_image_connection()
    if conn:
        cursor = conn.cursor()
        cursor.execute("INSERT INTO images (url) VALUES (?)", (url,))
        conn.commit()
        conn.close()

# Function to get saved images from the database
def get_saved_images():
    conn = get_image_connection()
    if conn is None:
        return []  # Return empty list if connection fails

    cursor = conn.cursor()
    cursor.execute("SELECT id, url FROM images")
    images = cursor.fetchall()
    conn.close()
    return images or []  # Return an empty list if no images found

# Sidebar initialization for saved images
if "saved_images" not in st.session_state:
    st.session_state.saved_images = [url for _, url in get_saved_images()]

# Temporary variable for storing edited post
if "temp_edited_post" not in st.session_state:
    st.session_state.temp_edited_post = None

# Main Page
st.image("Logo-transp.png", use_column_width=True)
st.title("Chatty - Post Generator")
st.write("Create a post for Linkedin!")

# Initialise chat history
if "messages" not in st.session_state:
    st.session_state.messages = []

if "images" not in st.session_state:
    st.session_state.images = []

# Display chat messages from history on app rerun
for idx, message in enumerate(st.session_state.messages):
    with st.chat_message(message["role"]):
        st.markdown(message["content"])
        if message["role"] == "assistant":
            # Add Edit and Save Post buttons
            col1, col2 = st.columns([1, 1])

            # Button for editing posts
            if col1.button("Edit Post", key=f"edit_{idx}"):
                st.session_state.edit_index = idx
                st.session_state.temp_edited_post = message["content"]

            # Display edit text area if post is being edited
            if st.session_state.temp_edited_post is not None and st.session_state.edit_index == idx:
                edited_text = st.text_area("Edit Post", value=st.session_state.temp_edited_post, key=f"edit_text_{idx}", height=200)
                st.session_state.temp_edited_post = edited_text

                # Button for saving the changes made after editing
                if st.button("Save Edited Post", key=f"save_changes_{idx}"):
                    # Update the message in session state
                    st.session_state.messages[idx]["content"] = st.session_state.temp_edited_post
                    save_post(st.session_state.temp_edited_post)

                    # Reset the temporary edited post variable
                    st.session_state.temp_edited_post = None
                    st.session_state.edit_index = None

                    st.success("Post saved to Saved Posts page.")

            # Button for saving the original post
            if col2.button("Save Post", key=f"save_{idx}"):
                # Save original post to the database
                save_post(message["content"])
                st.success("Post saved to Saved Posts page.")

# Display images in sidebar (if applicable)
st.sidebar.write("### Related Images")
for idx, images in enumerate(st.session_state.images):
    if images:
        st.sidebar.write(f"Images for Post {idx + 1}:")
        for img_idx, url in enumerate(images):
            st.sidebar.image(url, use_column_width=True)
            if st.sidebar.button("Add to Gallery", key=f"save_image_{idx}_{img_idx}"):
                # Save image URL to the session state and database
                st.session_state.saved_images.append(url)
                save_image_to_db(url)
                st.sidebar.success("Image added to Gallery.")

# React to user input
if prompt := st.chat_input("Create a post about..."):
    # Display user message in chat message container
    st.chat_message("user").markdown(prompt)

    # Add user message to chat history
    st.session_state.messages.append({"role": "user", "content": prompt})
    st.session_state.images.append([])  # Initialize an empty list for images for the user message

    # Begin spinner before answering question so it's there for the duration
    with st.spinner("Working on it..."):
        # Send question to chain to get answer
        answer = chain.invoke(prompt)
        response = answer.get("answer", "No response available.") if isinstance(answer, dict) else answer

        # Display chatbot response in chat message container
        with st.chat_message("assistant"):
            st.markdown(response)

        # Add assistant response to chat history
        st.session_state.messages.append({"role": "assistant", "content": response})

        # Add Edit and Save Post buttons immediately after generating response
        col1, col2 = st.columns([1, 1])

        # Button for editing the newly generated post
        if col1.button("Edit Post", key=f"edit_{len(st.session_state.messages) - 1}"):
            st.session_state.edit_index = len(st.session_state.messages) - 1
            st.session_state.temp_edited_post = response

        # Display edit text area if post is being edited
        if st.session_state.temp_edited_post is not None and st.session_state.edit_index == len(st.session_state.messages) - 1:
            edited_text = st.text_area("Edit Post", value=st.session_state.temp_edited_post, key=f"edit_text_{len(st.session_state.messages) - 1}", height=200)
            st.session_state.temp_edited_post = edited_text

            # Button for saving the changes made after editing
            if st.button("Save Edited Post", key=f"save_changes_{len(st.session_state.messages) - 1}"):
                st.session_state.messages[-1]["content"] = st.session_state.temp_edited_post
                save_post(st.session_state.temp_edited_post)

                # Reset the temporary edited post variable
                st.session_state.temp_edited_post = None
                st.session_state.edit_index = None

                st.success("Post saved to Saved Posts page.")

        # Button for saving the original response
        if col2.button("Save Post", key=f"save_{len(st.session_state.messages) - 1}"):
            save_post(response)
            st.success("Post saved to Saved Posts page.")

        # Extract hashtags from the response
        hashtags = extract_hashtags(response)

        # Fetch images based on the first 3 hashtags, if any
        image_urls = []
        if hashtags:
            for hashtag in hashtags[:3]:
                split_words = split_hashtag(hashtag)
                image_urls.extend(fetch_images_from_unsplash(split_words))
            image_urls = image_urls[:3]
            if not image_urls:
                st.warning("No images found to display.")
        else:
            st.warning("No hashtags found in the generated post to fetch images.")

        # Save images to session state
        st.session_state.images[-1] = image_urls

        # Display images in sidebar for the current post
        st.sidebar.write(f"### Images for Current Post:")
        for img_url in image_urls:
            st.sidebar.image(img_url, use_column_width=True)
            if st.sidebar.button("Add to Gallery", key=f"save_image_current_{img_url}"):
                # Save image URL to the session state and database
                st.session_state.saved_images.append(img_url)
                save_image_to_db(img_url)
                st.sidebar.success("Image added to Gallery.")

Overwriting 1_Home.py


In [ ]:
!streamlit run 1_Home.py


  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://192.168.1.53:8501

/Users/alexandrealbieri/Documents/Personal/Berlin/WBS Coding School/WBS Bootcamp/Data Science/Final Project/1_Home.py:3: LangChainDeprecationWarning: Importing FAISS from langchain.vectorstores is deprecated. Please replace deprecated imports:

>> from langchain.vectorstores import FAISS

with new imports of:

>> from langchain_community.vectorstores import FAISS
You can use the langchain cli to **automatically** upgrade many imports. Please see documentation here <https://python.langchain.com/docs/versions/v0_2/>
  from langchain.vectorstores import FAISS
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /Users/alexan